In [1]:
from pandas import read_excel

## Haltemagnete Schrack

In [49]:
file_name = "Schrack_HM_Obj420.xls"
columns = [
    "object",
    "level",
    "door_id",
    "door_id_alt",
    "bst_door_id_alt",
    "room_id",
    "aks_room_id",
    "uz6_control",
    "door_series",
    "lfd_id",
    "info"
]

hm_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="HALTEMAGNETE"
)

hm_data.columns = columns

# Add origin
hm_data["origin"] = file_name

hm_data.head(5)

,object,level,door_id,door_id_alt,bst_door_id_alt,room_id,aks_room_id,uz6_control,door_series,lfd_id,info,origin
0,420.0,EG,420 EG0A0106.02,004,420/0/3/6,001,0A0106,5.0,1.0,1.0,NaN,Schrack_HM_Obj420.xls
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Schrack_HM_Obj420.xls
2,420.0,EG,420 EG0A0803.02,040D,420/0/14/6,040D,0A0803,6.0,2.0,2.0,NaN,Schrack_HM_Obj420.xls
3,420.0,EG,420 EG0A0602.01,040A,420/0/15/6,040A,0A0602,6.0,2.0,3.0,NaN,Schrack_HM_Obj420.xls
4,420.0,EG,420 EG0A0406.01,008,420/0/22/6,004,0A0406,6.0,2.0,4.0,NaN,Schrack_HM_Obj420.xls


## Brandschutztechnik (BST) Sisando

In [50]:
file_name = "Sisando_BST_Obj420.xlsx"

columns = [
    "object",
    "level",
    "door_id",
    "status_text",
    "status",
    "door_class",
    "room_id",
    "room_id_additional",
    "door_audit_remark",
    "door_audit_date",
    "door_audit_responsible_short",
    "door_audit_responsible_long",
    "electrical_operation",
    "exf",
    "retention_mechanism",
    "manufacturer",
    "classification"
]

bst_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="Sheet1",
    parse_dates=[9]
)

bst_data.columns = columns

# Add origin
bst_data["origin"] = file_name

bst_data.head(5)

,object,level,door_id,status_text,status,door_class,room_id,room_id_additional,door_audit_remark,door_audit_date,door_audit_responsible_short,door_audit_responsible_long,electrical_operation,exf,retention_mechanism,manufacturer,classification,origin
0,420,0,A0201.01,OK,1,Drehflügel #1,A0201,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,NaN,NaN,Peneder,T30,Sisando_BST_Obj420.xlsx
1,420,0,A0203.01,OK,1,Drehflügel #2,A0203,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,1.0,NaN,Peneder,EI2-30-C,Sisando_BST_Obj420.xlsx
2,420,0,A0803.01,OK,1,Drehflügel #1,A0803,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,1.0,NaN,Peneder,EI2-30-C,Sisando_BST_Obj420.xlsx
3,420,0,A0803.02,HINWEIS,21,Drehflügel #2,A0803,NaN,STATUS OK_x001D_H000 Freilauftürschließer hält...,2024-10-28,peD,David PESCHEK,NaN,1.0,NaN,Fritscher,T30,Sisando_BST_Obj420.xlsx
4,420,0,A0807.03,OK,1,Drehflügel #2,A0807,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,NaN,NaN,Fritscher,T30,Sisando_BST_Obj420.xlsx


## Feuerleittechnik (FLT) Sisando

In [51]:
file_name = "Sisando_FLT_Obj420.xlsx"

columns = [
    "row_id",
    "object",
    "module",
    "section",
    "floor",
    "room_id",
    "number",
    "is_emergency_door",
    "is_emergency_door_no",
    "has_blind_cylinder",
    "has_cylinder",
    "has_panic_lock",
    "is_locked",
    "is_lockable",
    "is_not_lockable",
    "has_emergency_button",
    "has_emergency_button_no",
    "defect_memo",
    "defect_fixed",
    "none_a",
    "none_b",
    "none_c"
]

flt_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="all",
    parse_dates=[9],
    skiprows=[0,1],
    header=None,

)

flt_data.columns = columns

flt_data.drop("is_emergency_door_no", inplace=True, axis=1)
flt_data.drop("row_id", inplace=True, axis=1)
flt_data.drop("has_emergency_button_no", axis=1, inplace=True)
flt_data.drop("is_not_lockable", axis=1, inplace=True)
flt_data.drop("none_a", axis=1, inplace=True)
flt_data.drop("none_b", axis=1, inplace=True)
flt_data.drop("none_c", axis=1, inplace=True)

# Deleting of unused rows
flt_data["has_emergency_button"] = flt_data["has_emergency_button"].map(x_mapper)
flt_data["is_emergency_door"] = flt_data["is_emergency_door"].map(x_mapper)
flt_data["has_blind_cylinder"] = flt_data["has_blind_cylinder"].map(x_mapper)
flt_data["has_cylinder"] = flt_data["has_cylinder"].map(x_mapper)
flt_data["has_panic_lock"] = flt_data["has_panic_lock"].map(x_mapper)

# Transforming of yes/no rows
x_mapper = lambda x: True if x=="X" else False
flt_data["is_locked"] = flt_data["is_locked"].map(x_mapper)
flt_data["is_lockable"] = flt_data["is_lockable"].map(x_mapper)
flt_data["has_panic_lock"] = flt_data["has_panic_lock"].map(x_mapper)
flt_data["defect_fixed"] = flt_data["defect_fixed"].map(x_mapper)

# Other cleaning
object_floor_mapper = lambda x: x.split("_")[0]
flt_data["object"] = flt_data["object"].map(object_floor_mapper)

# Add origin
flt_data["origin"] = file_name

flt_data.head(5)

,object,module,section,floor,room_id,number,is_emergency_door,has_blind_cylinder,has_cylinder,has_panic_lock,is_locked,is_lockable,has_emergency_button,defect_memo,defect_fixed,origin
0,420,A,EG,A,3108,01,True,False,True,False,False,True,False,NaN,False,Sisando_FLT_Obj420.xlsx
1,420,A,EG,NaN,5206,05,False,False,True,False,False,True,False,"Panikschloss fehlt (Außentür, Offline-Beschlag?)",False,Sisando_FLT_Obj420.xlsx
2,420,A,EG,NaN,4401,07,True,False,False,False,False,False,False,NaN,False,Sisando_FLT_Obj420.xlsx
3,420,A,EG,NaN,2601,06,True,False,True,False,False,True,False,NaN,False,Sisando_FLT_Obj420.xlsx
4,420,A,EG,NaN,2601,04,True,False,True,False,False,True,False,NaN,False,Sisando_FLT_Obj420.xlsx
